# Spatiotemporal Trends in Urbanization
## Overview
This repository investigates year-by-year change in cities and settlements in Central and Western Africa (CWA). The goal is to capture activity for every settlement locality in a country to produce indicators that are high frequency, spatially granular, and timely. The Jupyter Notebook is the primary script used to construct each country's dataset. It tracks population, built-area, and economic and climate indicators across a 16-year timeframe from 2000 to 2015. 

The repository is split into three sections: methodology and notebooks, source data, and outputs. Outputs are organized by country and include growth tables ("urban panel datasets"), charts, and country briefs.


## Datasets
Datasets used to create each African country's urban panel data are as follows:
1. Most up-to-date administrative boundaries.
2. City names: **UCDB, Africapolis, and GeoNames.**
3. Settlement types: **GRID3 settlement extents.** Captured between 2009-2019.
4. Built-up area, yearly: **World Settlement Footprint Evolution.** Resolution: 30m.
5. Population, yearly: **WorldPop.** UN-adjusted, unconstrained. Resolution: 100m.
6. Nighttime lights, yearly: **Harmonization of DMSP and VIIRS.** Resolution: 1km and 500m.
7. Flood extents, by return period: **FATHOM.** Resolution: 90m.


## Accessing Data
Source data are available to the public by providers listed in the previous section, with the exception of flood data. Please note that the source data files in this repository have been fit for purpose and may not cover your area of interest. Some sources are also not global; GRID3 settlement extents are only available for sub-Saharan Africa, and Africapolis names for Africa.

Results from the analysis are currently available for Cameroon and are under development for Central African Republic and four Sahel countries: Burkina Faso, Chad, Mali, and Niger. Results are available in the outputs folder by country. Please contact the CWA Geospatial team to inquire about new locations.
<br>
> **Walker Kosmidou-Bradley**, wkosmidoubradley@worldbank.org
<br>
> **Grace Doherty**, gdoherty2@worldbank.org

## License
Materials under this repository are open-source under an MIT license. The community is invited to test, adapt, and re-purpose materials as needed.

---

## 1. PREPARE WORKSPACE

### 1.1 Off-script

##### Off-script: Create folders in your working directory. (The folder where you are storing this script).
> *ADM
<br>Buildup
<br>PlaceName
<br>Population
<br>Settlement
<br>NTL*

##### Before starting: Download datasets (as shapefile, GeoJSON, or tif where possible) and place or extract into corresponding folder. You can download the cleaned files from our [GitHub Repository](https://github.com/worldbank/Urban_Spatio_Temporal_Trends) or access original sources here:
- ADM: *Varies by source.*
- Buildup: https://download.geoservice.dlr.de/WSF_EVO/files/
    - If more than one tif, name the tifs as follows: WSFE1.tif, WSFE2.tif... etc.
- PlaceName: 
    - GeoNames: (file: cities500.zip) https://download.geonames.org/export/dump/
    - Africapolis: https://africapolis.org/en/data
    - Urban Centres Database: https://ghsl.jrc.ec.europa.eu/ghs_stat_ucdb2015mt_r2019a.php
- Population: https://hub.worldpop.org/geodata/listing?id=69
- Settlement: https://data.grid3.org/datasets/GRID3::grid3-cameroon-settlement-extents-version-01-01-/explore
- Nighttime Lights: https://eogdata.mines.edu/products/dmsp/#v4 and https://eogdata.mines.edu/products/vnl/#annual_v2

##### Other off-script:
- Convert GeoNames from .txt file to shape (delimiter = tab, header rows = 0) and rename fields.
- If necessary, mosaic WSFE rasters that cover the area of interest to create a single file.

### 1.2 Load all packages.

In [ ]:
# Built-in:
# dir(), print(), range(), format(), int(), len(), list(), max(), min(), zip(), sorted(), sum(), open(), del, = None, try except, with as, for in, if elif else
# Also: list.append(), list.insert(), list.remove(), count(), startswith(), endswith(), contains(), replace()

import os, sys, glob, re, time, subprocess, string # os.getcwd(), os.path.join(), os.listdir(), os.remove(), time.ctime(), glob.glob(), string.zfill(), string.join()
from os.path import exists # exists()
from functools import reduce # reduce()

import geopandas as gpd # read_file(), GeoDataFrame(), sjoin_nearest(), to_crs(), to_file(), .crs, buffer(), dissolve()
import pandas as pd # .dtypes, Series(), concat(), DataFrame(), read_table(), merge(), to_csv(), .loc[], head(), sample(), astype(), unique(), rename(), between(), drop(), fillna(), idxmax(), isna(), isin(), apply(), info(), sort_values(), notna(), groupby(), value_counts(), duplicated(), drop_duplicates()
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid  # in apply(make_valid)
import shapely.wkt

import numpy as np # median(), mean(), tolist(), .inf
import fiona, rioxarray # fiona.open()
import rasterio # open(), write_band(), .name, .count, .width, .height. nodatavals, .meta, update(), copy(), write()
from rasterio.plot import show
from rasterio import features # features.rasterize()
from rasterio.features import shapes
from rasterio import mask # rasterio.mask.mask()
from rasterio.enums import Resampling # rasterio.enums.Resampling()
from rasterstats import zonal_stats # zonal_stats()
from osgeo import gdal, osr, ogr, gdal_array, gdalconst # Open(), SpatialReference, WarpOptions(), Warp(), GetDataTypeName(), GetRasterBand(), GetNoDataValue(), Translate(), GetProjection(), GetAttrValue()

### 1.3 Set up workspace.

In [ ]:
Workspace = os.getcwd()
Source = os.path.join(Workspace, 'Source')
Intermediate = os.path.join(Workspace, 'Intermediate')
Results = os.path.join(Workspace, 'Results')

print('\n'.join([Workspace, Source, Intermediate, Results]))

In [ ]:
def ListFromRange(r1, r2):
    return [item for item in range(r1, r2+1)]

In [ ]:
WSFEYears = ListFromRange(1985, 2015) # All years in the WSFE dataset.
AllStudyYears = ListFromRange(1999, 2015) # All years for which there will be growth stats in the present study.
ReversedStudyYears = []
for i in AllStudyYears:
    ReversedStudyYears.insert(0,i)
print(WSFEYears, '\n\n', AllStudyYears, '\n\n', ReversedStudyYears)

### 1.4 User-defined functions.

In [ ]:
# From Stack Exchange @RutgerH
# https://gis.stackexchange.com/questions/163685/reclassify-a-raster-value-to-9999-and-set-it-to-the-nodata-value-using-python-a
def readRaster(filename):
    filehandle = gdal.Open(filename)
    band1 = filehandle.GetRasterBand(1)
    geotransform = filehandle.GetGeoTransform()
    geoproj = filehandle.GetProjection()
    Z = band1.ReadAsArray()
    xsize = filehandle.RasterXSize
    ysize = filehandle.RasterYSize
    return xsize,ysize,geotransform,geoproj,Z

In [ ]:
# Default arguments can be changed here, or can be specified below when running the functions.
def writeRaster(filename,geotransform,geoprojection,data, NoDataVal=0, dst_datatype=gdal.GDT_UInt32):
    (x,y) = data.shape
    Dformat = "GTiff"
    driver = gdal.GetDriverByName(Dformat)
    # you can change the dataformat but be sure to be able to store negative values including -9999
    dst_ds = driver.Create(filename,y,x,1,dst_datatype)
    dst_ds.GetRasterBand(1).WriteArray(data)
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(geoprojection)
    dst_ds.GetRasterBand(1).SetNoDataValue(NoDataVal)
    return 1
    dst_ds = None

In [ ]:
# Based on Stack Exchange @Kurt Schwehr:
# https://stackoverflow.com/questions/10454316/how-to-project-and-resample-a-grid-to-match-another-grid-with-gdal-python
def resampleRaster(InRaster_Path, MatchRaster_Path, OutFile_Path, 
                   DataType = gdalconst.GDT_UInt32, 
                   ResampType = gdal.GRA_Bilinear, NoDataVal = 0):
    print('Loading for %s. %s' % (InRaster_Path, time.ctime()))
    
    RasterObject = gdal.Open(InRaster_Path)
    In_proj = RasterObject.GetProjection()
    [Match_x, Match_y, Match_geo, Match_proj, Match_Z] = readRaster(MatchRaster_Path)
    print('---Specs to match to: \n', 
      Match_proj, '\n', Match_geo, '\n', Match_x, '\n', Match_y, '\n')
        
    OutFile = gdal.GetDriverByName('GTiff').Create(OutFile_Path, Match_x, Match_y, 1, DataType)
    OutFile.SetGeoTransform(Match_geo)
    OutFile.SetProjection(Match_proj)
    print('---Created raster file for upsampled version. %s' % time.ctime())
    
    gdal.ReprojectImage(RasterObject, OutFile, In_proj, Match_proj, ResampType)
    print('---Resampled values onto an empty raster matching the dimensions of the buildup layer. %s \n\n' % time.ctime())
    
    OutFile.GetRasterBand(1).SetNoDataValue(NoDataVal)
    
    RasterObject = Outfile = None
    return 1

In [ ]:
def calcShell(A, OutFile, Calculation, OutType = '', 
              B=None, C=None, D=None, E=None, F=None, G=None):
    """Raster math using gdal_calc.py.

    The OSgeo package for Python API does not make raster calculations
    easy outside of the shell. This function plugs up to 6 raster files
    into a string which subprocess.call() then commits to the terminal.

        A : str
            File path to the first raster for the calculation.
        B : str
            File path to the second raster for the calculation.
        OutFile : str
            File path where to store the raster generated from the calculation.
        Calculation : str
            Algebra that uses A and B to create a new raster. Use double quotes.
    """
    print('Running for %s. %s' % (A, time.ctime()))
    cmd = 'gdal_calc.py -A ' + A
    if B is not None:
        cmd = cmd + ' -B ' + B 
    if C is not None:
        cmd = cmd + ' -C ' + C 
    if D is not None:
        cmd = cmd + ' -D ' + D
    if E is not None:
        cmd = cmd + ' -E ' + E
    if F is not None:
        cmd = cmd + ' -F ' + F
    if G is not None:
        cmd = cmd + ' -G ' + G
    cmd = cmd + OutType + ' --outfile=' + OutFile + ' --overwrite --calc=' + Calculation
    subprocess.call(cmd, shell=True)
    cmd = A = B = C = D = E = F = G = None
    print('Ran in shell. See OutFile folder to inspect results. %s' % time.ctime())

In [ ]:
def mosaicShell(A, B, OutFile, Band = 1, OutType = '',
                  C=None, D=None, E=None, F=None, G=None):
    print('Running for %s. %s' % (A, time.ctime()))
    
    StringFiles = ' '.join([A,B])
    
    for RasterName in [C,D,E,F,G]:
        if RasterName is not None:
            StringFiles = ' '.join([StringFiles, RasterName])
        else:
            pass
        
    cmd = 'gdal_merge.py -o ' + OutFile + OutType + ' -of gtiff ' + StringFiles
    
    subprocess.call(cmd, shell=True)
    print('Ran in shell. See OutFile folder to inspect results. %s' % time.ctime())

In [ ]:
def RasterToShapefile(InRasterPath, OutFilePath = 'RastToShp.shp', Band=1, 
                      OutName='RastToShp', VariableName='value', Driver = 'ESRI Shapefile'):
    """Raster tiff to vector polygon shapefile.
    Can also be used for other file types like geopackage, but note that this code
    currently does not account for writing into an existing file. It will write over
    the file if specified as the file path.
    
    """
    Raster = gdal.Open(InRasterPath)
    RasterBand = Raster.GetRasterBand(Band)
    
    OutDriver = ogr.GetDriverByName(Driver)
    InProj = Raster.GetProjectionRef()
    SpatRef = osr.SpatialReference()
    SpatRef.ImportFromWkt(InProj)
    print(InProj, '\n\n', SpatRef)
    
    if exists(OutFilePath):
        OutFile = ogr.Open(OutFilePath)
    else:
        OutFile = OutDriver.CreateDataSource(OutFilePath)
    OutLayer = OutFile.CreateLayer(OutName, srs = SpatRef, geom_type = ogr.wkbPolygon)
    OutField = ogr.FieldDefn(VariableName, ogr.OFTInteger)
    OutLayer.CreateField(OutField)
    OutField = OutLayer.GetLayerDefn().GetFieldIndex(VariableName)
    print('\n', OutFile, '\n', OutLayer, '\n', OutField)
    
    print('Vectorizing. Input: %s. %s' % (InRasterPath, time.ctime()))
    gdal.Polygonize(RasterBand, None, OutLayer, 0, [], callback=None)
    print('Completed polygons. Stored as: %s. %s' % (OutFilePath, time.ctime()))

    del Raster, RasterBand, OutFile, OutLayer

In [ ]:
def rioStats(InRasterPath, Band = 1):
    out = rasterio.open(InRasterPath)
    stats = []
    band = out.read(Band)
    stats.append({
        'raster': out.name,
        'bands': out.count,
        'data type': out.dtypes,
        'no data value': out.nodatavals,
        'width': out.width,
        'height': out.height,
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})
    print("\n", stats)
    
    out = band = None

In [ ]:
def ShapeToRaster(Shapefile, ValueVar, MetaRasterPath, OutFilePath = 'ShpToRast.tif', Band=1, NewDType=None):
    """
    Polygon spatial object to raster tiff.
    """
    # Copy and update the metadata from another raster for the output
    MetaRaster = rasterio.open(MetaRasterPath)
    meta = MetaRaster.meta.copy()
    meta.update(compress='lzw')
    if NewDType is not None:
        meta.update(dtype=NewDType)
    MetaRaster.meta

    print("Rasterizing dataset. %s" % time.ctime())
    with rasterio.open(OutFilePath, 'w+', **meta) as out:
        out_arr = out.read(Band)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(Shapefile.geometry, Shapefile[ValueVar]))

        burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
        out.write_band(1, burned)
    out = burned = shapes = None
    
    print("Finished rasterizing. Checking contents. %s" % time.ctime())
    rioStats(OutFilePath)

In [ ]:
def BatchZonal(RasterFileList, Zones, KeepFields = None, 
               RasterDirectory = os.getcwd(),
               OutPath = 'BatchZonal.csv',
               Statistics=['count', 'sum', 'mean'],
               NoDataVal = None,
               Prefix = '',
               Suffix = '', 
               DropStatName = False):
    '''
    Choose a single file for the raster source, or multiple. If multiple, then each stat field
    will receive a suffix with the raster's position in the RasterFileList.
    
        RasterFileList: list
                        List of file paths, e.g. tifs. Loaded with rasterio.
        Zones:          geodataframe object
        KeepFields:     list
                        List of field names (string format).
        OutPath:        string
                        File path for the results.
        Statistics:     list
                        Full Statistics options: 
                        ['max', 'min', 'mean', 'count', 'sum', 'median', 'std', 'majority', 'minority', 
                        'unique', 'range']
                        There's also a percentile option in zonal_stats(). I believe you 
                        write it after an underscore in string format, e.g.: 'percentile_25'.
        NoDataVal:      numeric
        Prefix:         string
        Suffix:         string
        DropStatName:   boolean
    
    '''
    if KeepFields is None:
        AllSummaries = pd.DataFrame(Zones)[[]]
    else:
        AllSummaries = pd.DataFrame(Zones)[KeepFields]
    print('Dataframe to merge with: \n', AllSummaries)
    
    
    for idx, RasterFile in enumerate(RasterFileList):
        print('Loading with rasterio. %s\nRaster: %s' % (time.ctime(), RasterFile))
        
        InRasterPath = os.path.join(RasterDirectory, RasterFile)
        
        Year = re.search('\d{4}', os.path.basename(RasterFile))
        if Year is None:
            Year = '_' + str(idx)
        else:
            Year = Year.group(0)

        with rasterio.open(InRasterPath) as src:
            transform = src.meta['transform']
            array = src.read(1)

        print(transform)
        print(array)
        
        print('Zonal statistics. %s' % time.ctime())
        zStats = zonal_stats(Zones, array, affine=transform, stats = Statistics, nodata=NoDataVal)
        #print(zStats)

        ValByZone = pd.DataFrame(Zones)[[]].join(pd.DataFrame(zStats))
        
        if DropStatName is False:
            for stat in Statistics:
                StatField = ''.join([Prefix, stat, Year, Suffix])
                ValByZone = ValByZone.rename(columns={stat:StatField})
            print('%s stat output field for %s: %s' % (stat, RasterFile, StatField))
        else:
            for stat in Statistics:
                StatField = ''.join([Prefix, Year, Suffix])
                ValByZone = ValByZone.rename(columns={stat:StatField})
            print('%s stat output field for %s: %s' % (stat, RasterFile, StatField))

        AllSummaries = AllSummaries.join(ValByZone)
        print(AllSummaries.sample(5))
        
    print('Final dataframe: \n', AllSummaries.sample(5))

    AllSummaries.to_csv(OutPath)

In [ ]:
def MaskByZone(MaskPath, SourceFolder, DestFolder, SourceList = None,
               MaskLayerName = None, dstSRS = 'ESRI:102022'):
    """
    Reduces the size of a raster's valid data cells to vector areas of interest.
    This is useful if the raster data needs to be vectorized later to save space.
    
    The script prepares the vector zones as a list of geometries in the desired
    spatial reference system, then warps each raster in the specified source
    folder to the same SRS. Masking in rasterio then reclassifies any raster cells
    falling outside of a mask polygon as NoData.
    """
    
    ProjSRS = osr.SpatialReference()
    ProjSRS.SetFromUserInput(dstSRS)
    ProjWarp = gdal.WarpOptions(dstSRS = dstSRS)
    
    if SourceList is not None:
        SourceFiles = SourceList
    else:
        SourceFiles = []
        SourceFiles = SourceFiles + [i for i in os.listdir(''.join([SourceFolder, r'/'])) if i.endswith('tif')]
        print(SourceFiles)

    
    ### 1. ASSIGN SPATIAL REFERENCE SYSTEM OF VECTOR MASK AND LOAD GEOMETRIES
    Vector = gpd.read_file(filename=MaskPath, layer=MaskLayerName)
    if Vector.crs != dstSRS:
        if MaskLayerName == None:
            MaskPath = MaskPath + '_temp'
        else:
            MaskLayerName = MaskLayerName + '_temp'
        Vector.to_crs(dstSRS).to_file(filename=MaskPath, layer=MaskLayerName)
    Vector = None # We're reloading the geometries with fiona
    
    with fiona.open(MaskPath, mode="r", layer=MaskLayerName) as Vector:
        MaskGeom = [feature["geometry"] for feature in Vector] # Identify the bounding areas of the mask.
    
    
    ### 2. PREPARE DESTINATION FILES
    for FileName in SourceFiles:
    
        InputRasterPath = os.path.join(Workspace, SourceFolder, FileName)
        
        Sensor = re.search('[A-Z]+_', FileName)
        if Sensor is None:
            Sensor = ''
        else:
            Sensor = Sensor.group(0)

        Year = re.search('\d{4}', FileName)
        if Year is None:
            Year = ''
        else:
            Year = Year.group(0)

        if FileName.endswith('avg.tif') == True:
            IndicType = '_avg'
        elif FileName.endswith('cfc.tif') == True:
            IndicType = '_cfc'
        else:
            IndicType = ''

        TempOutputName = 'Temp_' + Sensor + Year + IndicType + '.tif'
        TempOutputPath = os.path.join(Workspace, DestFolder, TempOutputName)
        FinalOutputName = 'Msk_' + Sensor + Year + IndicType + '.tif'
        FinalOutputPath = os.path.join(Workspace, DestFolder, FinalOutputName)

    ### 3. ASSIGN SPATIAL REFERENCE SYSTEM OF RASTER(S)
        InputRasterObject = gdal.Open(InputRasterPath)
        SourceSRS = osr.SpatialReference(wkt=InputRasterObject.GetProjection())
        print('Source projection: ', SourceSRS.GetAttrValue('projcs'))
        print('Destination projection: ', ProjSRS.GetAttrValue('projcs'))

        if SourceSRS.GetAttrValue('projcs') != ProjSRS.GetAttrValue('projcs'):
            Warp = gdal.Warp(TempOutputPath, # Where to store the warped raster
                         InputRasterObject, # Which raster to warp
                         format='GTiff', 
                         options=ProjWarp) # Reproject to Africa Albers Equal Area Conic
            print('Finished gdal.Warp() for %s. %s \n' % (FileName, time.ctime()))

            Warp = None # Close the files
        else:
            pass
        InputRasterObject = None
        
    ### 4. RECLASSIFY AS NODATA IF OUTSIDE OF SETTLEMENT BUFFER ZONE.
        if exists(TempOutputPath):
            NewInputPath = TempOutputPath 
            print("We warped the data, so we'll use that file for next step.")
        else:
            NewInputPath = InputRasterPath 
            print("We skipped the warp, so we continue to use the source file.")

        with rasterio.open(NewInputPath) as InputRasterObject:
            MaskedOutputRaster, OutTransform = rasterio.mask.mask(
                InputRasterObject, MaskGeom, crop=True) # Anything outside the mask is reclassed to the raster's NoData value.
            OutMetaData = InputRasterObject.meta.copy()
        print('Finished rasterio.mask.mask() for %s. %s \n' % (FileName, time.ctime()))

        OutMetaData.update({"driver": "GTiff",
                         "height": MaskedOutputRaster.shape[1],
                         "width": MaskedOutputRaster.shape[2],
                         "transform": OutTransform})

        with rasterio.open(FinalOutputPath, "w", **OutMetaData) as dest:
            dest.write(MaskedOutputRaster)
        print('Written to file. %s \n' % time.ctime())
        InputRasterObject = None

        if exists(TempOutputPath):
            try:  # Finally, remove the intermediate file from disk
                os.remove(TempOutputPath)
            except OSError:
                pass
            print('Removed intermediate file. %s \n' % time.ctime())
        else:
            pass


    print('\n \n Finished all years in list. %s' % time.ctime())

In [ ]:
def BatchZonalStats(FolderName, Zones, 
                    CRS = 'ESRI:102022', 
                    JoinField = 'Sett_ID',
                    StatsWanted = ['count', 'sum', 'mean', 'max', 'min'],
                    SeriesStart = 1999, SeriesEnd = 2015, 
                    AnnualizedFiles = None, VarPrefix = None):
    """
    Normally, we would use numpy to generate a point gdf from the raster's matrix. 
    However, I was running into a lot of memory errors with that method.
    This method uses some extra steps: tif to xyz to df to gdf. But it saves to file
    and deletes intermediate files along the way, circumventing memory issues.
    
    Run MaskByZone() prior to reduce the raster to only your area(s) of interest.
    
    """
    if AnnualizedFiles is None:
        AnnualizedFiles = [i for i in os.listdir(FolderName) if i.endswith('.tif')]
    print(AnnualizedFiles)
    AllSummaries = pd.DataFrame(Zones).drop(columns='geometry')[[JoinField]]
    print(AllSummaries)
    
    if VarPrefix is None:
        VarPrefix = FolderName[:3].upper()
    
    for FileName in AnnualizedFiles:
    ### STEP 1: TIF TO XYZ ###
        print('Loading data for %s. %s \n' % (FileName, time.ctime()))
        
        Sensor = re.search('[A-Z]+_', FileName)
        if Sensor is None:
            Sensor = ''
        else:
            Sensor = Sensor.group(0)
            
        Year = re.search('\d{4}', FileName)
        if Year is None:
            Year = ''
        else:
            Year = Year.group(0)
        
        InputRasterPath = os.path.join(Workspace, FolderName, FileName)
        InputRasterObject = gdal.Open(InputRasterPath)
        XYZOutputPath = FolderName + r'/{}'.format(
            FileName.replace('.tif', '.xyz')) # New file path will be the same as original, but .tif is replaced with .xyz

        # Create an .xyz version of the .tif
        if exists(XYZOutputPath):
            print("Already created xyz file.")
        else:
            print("Creating XYZ (gdal.Translate()).")
            XYZ = gdal.Translate(XYZOutputPath, # Specify a destination path
                                 InputRasterObject, # Input is the masked .tif file
                                 format='XYZ', 
                                 creationOptions=["ADD_HEADER_LINE=YES"])
            print('Finished gdal.Translate() for year %s. %s \n' % (Year, time.ctime()))
            XYZ = None # Reload XYZ as a point geodataframe

        InputRasterObject = None


    ### STEP 2: GENERATE GEODATAFRAME WITH JOIN FIELD ###
        InputXYZ = pd.read_table(XYZOutputPath, delim_whitespace=True)
        InputXYZ = InputXYZ.loc[InputXYZ['Z'] > 0] # Subset to only the features that have a value.
        
        if re.search('WSFE', FileName) is not None: # Scale back up to years if working with flood/building data.
            InputXYZ['Z'] = InputXYZ['Z'] + 1900
            
        print('Loaded XYZ file as a pandas dataframe. %s \n' % time.ctime())
        ValObject = gpd.GeoDataFrame(InputXYZ,
                                     geometry = gpd.points_from_xy(InputXYZ['X'], InputXYZ['Y']),
                                     crs = CRS)
        print('Created geodataframe from non-NoData points. %s \n' % time.ctime())
        del InputXYZ

        # Sjoin_nearest: No need to group by ADM this time. 
        ValObject_withID = pd.DataFrame(gpd.sjoin_nearest(ValObject, 
                                        Zones, 
                                        how='left')).drop(columns='geometry')[['Z', JoinField]] # No need for max_distance parameter this time. We've already narrowed down to nearby raster cells.

        print('\nJoined zone ID onto vectorized raster cells. %s \n' % time.ctime())
        print(ValObject_withID.sample(10))
        del ValObject

        ValObject_withID.to_csv(''.join([FolderName, r'/', FileName.replace('.tif', '.csv')]))
        print('\nExported as table. %s \n' % time.ctime())

#         # Remove the temporary xyz file.
#         try:  
#             os.remove(os.path.join(XYZOutputPath))
#         except OSError:
#             pass
#         print('Removed (or skipped if error) intermediate xyz file. %s \n' % time.ctime())


    ### STEP 3: AGGREGATE BY SETTLEMENT AND MERGE ONTO SUMMARIES TABLE ###
        GroupedVals = ValObject_withID[ValObject_withID['Z'].notna()].groupby(JoinField, as_index=False)
        
        # Run this block if the variable is about cloud-free coverage.
        if re.search('cfc', FileName) is not None:
            VariableName = ''.join([VarPrefix, 'cfc_', Sensor, Year])
            AllSummaries = AllSummaries.merge(GroupedVals.mean().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            print('\nCount of cloud-free observations averaged to settlement level, year %s. %s \n' % (Year, time.ctime()))
            
            # Save in-progress results
            AllSummaries.to_csv(os.path.join(Results, ''.join([VarPrefix, '%sto%s.csv' % (SeriesStart, SeriesEnd)])))
            print(AllSummaries.sample(10))
        
        # Run this block if we're working with the flooded buildings data.
        elif re.search('WSFE', FileName) is not None:
            for BuiltYear in AllStudyYears:
                Grouped_Subset = GroupedVals[GroupedVals['Z'].between(
                    1985, BuiltYear, inclusive=True)] # Inclusive parameter means we include the years 1985 and the named year.
                if 'count' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'ct', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.count().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                if 'sum' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'sum', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.sum().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                if 'mean' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'avg', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.mean().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                if 'max' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'max', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.max().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                if 'min' in StatsWanted:
                    VariableName = ''.join([VarPrefix, 'min', Sensor, BuiltYear])
                    AllSummaries = AllSummaries.merge(GroupedVals.min().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
                print('\nDesired aggregation methods applied to settlement level, year %s. %s \n' % (Year, time.ctime()))

                # Save in-progress results
                AllSummaries.to_csv(os.path.join(Results, ''.join([VarPrefix, '%sto%s.csv' % (SeriesStart, SeriesEnd)])))
                print(AllSummaries.sample(10))
        
        # Anything else takes the standard aggregation method.
        else:
            if 'count' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'ct', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.count().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            if 'sum' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'sum', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.sum().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            if 'mean' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'avg', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.mean().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            if 'max' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'max', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.max().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            if 'min' in StatsWanted:
                VariableName = ''.join([VarPrefix, 'min', Sensor, Year])
                AllSummaries = AllSummaries.merge(GroupedVals.min().rename(columns={'Z': VariableName}), how = 'left', on=JoinField)
            print('\nDesired aggregation methods applied to settlement level, year %s. %s \n' % (Year, time.ctime()))
            
            # Save in-progress results
            AllSummaries.to_csv(os.path.join(Results, ''.join([VarPrefix, '%sto%s.csv' % (SeriesStart, SeriesEnd)])))
            print(AllSummaries.sample(10))

    
    print('\n\nFinished. All years masked and assigned their nearest settlement. %s' % time.ctime())
    print(AllSummaries.sample(10))
    AllSummaries.to_csv(os.path.join(Results, ''.join([VarPrefix, '%sto%s.csv' % (SeriesStart, SeriesEnd)])))
    print('Saved to file. %s \n' % time.ctime())

---

## 2. PREPARE BUILDUP, SETTLEMENT, AND ADMIN DATASETS
Projection for all datasets: Africa Albers Equal Area Conic

### 2.1 Prepare GRID3 and Admin area files

#### Admin areas

In [ ]:
# Pull the first file ([0]) which ends in '.shp' from the specified folder, drop all variables, and reproject.
ADM_vec = gpd.read_file(glob.glob('Source/ADM/*.shp')[0])[['geometry']].to_crs('ESRI:102022')

# Create a fresh unique ID
ADM_vec['ADM_ID'] = range(0, len(ADM_vec))
ADM_vec['ADM_ID'] = ADM_vec['ADM_ID'] + 1 
# We have to add 1 if we want our rasterized version's NoData value to be 0. Otherwise the first feature won't be valid.
ADM_vec.to_file(driver='GPKG', filename=os.path.join(Source, 'ADM', 'ADM_withID.gpkg'), layer='ADM')

# Now reload.
ADM_vec = gpd.read_file(os.path.join(Source, 'ADM', 'ADM_withID.gpkg'), layer='ADM')

In [ ]:
# We need to know how many digits need to be allocated to each dataset in the "join" serial.
len_ADM = len(str(ADM_vec['ADM_ID'].max()))

print(ADM_vec.info(), "\n\n", 
      ADM_vec.sample(5),
      ADM_vec.crs, "\n\n", 
      'Number of digits: ', len_ADM) 

#### Settlements

In [ ]:
# For GRID3, we will still retain the 'type' field.
GRID3_vec = gpd.read_file(glob.glob('Source/Settlement/*.shp')[0])[['type','geometry']].to_crs("ESRI:102022")

GRID3_vec['G3_ID'] = range(0,len(GRID3_vec))
GRID3_vec['G3_ID'] = GRID3_vec['G3_ID'] +1

GRID3_vec.to_file(driver='GPKG', filename=os.path.join(Source, 'Settlement', 'Settlement_withID.gpkg'), layer='GRID3')

GRID3_vec = gpd.read_file(os.path.join(Source, 'Settlement', 'Settlement_withID.gpkg'), layer='GRID3')

In [ ]:
len_G3 =  len(str(GRID3_vec['G3_ID'].max()))

print(GRID3_vec.info(), "\n\n",
      GRID3_vec.sample(5),
      GRID3_vec.crs, "\n\n", 
      'Number of digits: ', len_G3)

### 2.2 Reproject WSFE to project CRS, and ensure valid data range is only 1985-2015.

In [ ]:
if exists(os.path.join(Source, 'Buildup', 'WSFE.tif')):
    pass
else:
    # If WSFE hasn't been mosaicked yet:
    A=os.path.join(Source, 'Buildup', 'Premosaic', 'WSFE1.tif')
    B=os.path.join(Source, 'Buildup', 'Premosaic', 'WSFE2.tif')
    C=os.path.join(Source, 'Buildup', 'Premosaic', 'WSFE3.tif')
    D=os.path.join(Source, 'Buildup', 'Premosaic', 'WSFE4.tif')

    Out=os.path.join(Intermediate, 'Buildup', 'WSFE.tif')

    mosaicShell(A=A, B=B, C=C, D=D, OutFile=Out, OutType = ' -ot UInt32 ')

    Out=None

In [ ]:
InPath = glob.glob('Source/Buildup/*.tif')[0]
OutWGS = os.path.join(Intermediate, 'WSFE_reclass.tif')

# Together, x and y define the data's "shape".
# geotransform contains the parameters detailing how the raster should be stretched and aligned.
# geoproj is the map projection
# Z are the values in the raster band.
[xsize,ysize,geotransform,geoproj,Z] = readRaster(InPath)
Z[Z<1985] = 0
Z[Z>2015] = 0

writeRaster(OutWGS,geotransform,geoproj,Z, NoDataVal=0, dst_datatype=gdal.GDT_UInt32)
print('Wrote the reclassed raster to file. %s' % time.ctime())

In [ ]:
OutEqArea = os.path.join(Intermediate, 'WSFE_equalarea.tif')

# Whenever we want to work in a projected CRS, we'll use Africa Albers Equal Area Conic.
ProjEqArea = gdal.WarpOptions(dstSRS='ESRI:102022')
Warp = gdal.Warp(OutEqArea, # Where to store the warped raster
                 OutWGS, # Which raster to warp
                 format='GTiff', 
                 options=ProjEqArea)
print('Wrote the reclassed and reprojected raster to file. %s' % time.ctime())

In [ ]:
rioStats(OutWGS)
rioStats(OutEqArea)

In [ ]:
InPath = Warp = OutWGS = OutEqArea = None

---

## 3. WSFE AND ADM; GRID3 AND ADM
RASTERIZE: Bring ADM and GRID3 into raster space.

RASTER MATH: "Join" ADM ID onto GRID3 and onto WSFE by creating unique concatenation string.

VECTORIZE: Bring joined data into vector space.

VECTOR MATH: Split unique ID from raster math step into separate columns.

### 3.1 Rasterize admin areas and GRID3 using WSFE specs.

In [ ]:
# Copy and update the metadata from WSFE for the output
WSFE = os.path.join(Intermediate, 'WSFE_equalarea.tif')
ADM_out = os.path.join(Intermediate, 'ADM_rasterized.tif')
GRID3_out = os.path.join(Intermediate, 'GRID3_rasterized.tif')

In [ ]:
# ADM raster can be unsigned int 16. Consider 32 if there are thousands of admin areas.
ShapeToRaster(Shapefile=ADM_vec, ValueVar="ADM_ID", MetaRasterPath=WSFE, OutFilePath=ADM_out, NewDType = 'uint16')

# To be safe, we'll do unsigned 32 for settlements.
ShapeToRaster(GRID3_vec, "G3_ID", WSFE, GRID3_out, NewDType='uint32')

# Check printed stats here to make sure the min is 0 and the max is the number of vector features in the input.

### 3.2 Raster math to "join" admin to GRID3 and to WSFE.
Processing is more rapid when "joining," i.e. creating serial codes out of two datasets, in raster rather than vector space.
Here, we are concatenating the ID fields of the two datasets to create a serial number that we can then split in vector space later to create two ID fields.

In [ ]:
# In paths
InG3 = os.path.join(Intermediate, 'GRID3_rasterized.tif')
InWSFE = os.path.join(Intermediate, 'WSFE_equalarea.tif')
InADM = os.path.join(Intermediate, 'ADM_rasterized.tif')

# Out paths
G3_ADM = os.path.join(Intermediate, 'GRID3_ADM.tif')
WSFE_ADM = os.path.join(Intermediate, 'WSFE_ADM.tif')

In [ ]:
G3_rio = rasterio.open(InG3).read(1)
WSFE_rio = rasterio.open(InWSFE).read(1)
ADM_rio = rasterio.open(InADM).read(1)

# Recalculate number of digits for each dataset if starting from Section 3
len_G3 = len(str(G3_rio.max()))
len_WSFE = len(str(WSFE_rio.max()))
len_ADM = len(str(ADM_rio.max()))

G3_rio = WSFE_rio = ADM_rio = None
print('DIGITS', '\nGRID3: ', len_G3, '\nWSFE: ', len_WSFE, '\nADM: ', len_ADM)

In [ ]:
# Calculations
# The number of digits in the largest ADM index value (len_ADM) is 
# the number of zeroes we tack onto the first variable in the serial.

Calc = "(A*" + str(10**len_ADM) + ")+B" 

calcShell(A=InG3, B=InADM, OutFile=G3_ADM, Calculation=Calc)
calcShell(A=InWSFE, B=InADM, OutFile=WSFE_ADM, Calculation=Calc)

*Adding together the values to create join IDs. This is in effect a concatenation of their ID strings, by way of summation. The number of zeros in the calc multiplication corresponds with number of digits of the maximum value in the "B" dataset. (e.g. Chad ADM codes go up 4 digits, so it's calc=(A*10000)+B).*

In [ ]:
rioStats(G3_ADM)
rioStats(WSFE_ADM)

### 3.3 Vectorize serialized layers.

In [ ]:
G3_in = os.path.join(Intermediate, 'GRID3_ADM.tif')
G3_out = os.path.join(Intermediate, 'GRID3_ADM.shp')
WSFE_in = os.path.join(Intermediate, 'WSFE_ADM.tif')
WSFE_out = os.path.join(Intermediate, 'WSFE_ADM.shp')

In [ ]:
RasterToShapefile(G3_in, G3_out, OutName='GRID3_ADM', VariableName='gridcode', Driver = 'ESRI Shapefile')

In [ ]:
RasterToShapefile(WSFE_in, WSFE_out, OutName='WSFE_ADM', VariableName='gridcode', Driver = 'ESRI Shapefile')

### 3.4 Vector math to split raster strings into admin area, GRID3, and WSFE year assignments.

In [ ]:
# Load newly created vectorized datasets.
GRID3_ADM = gpd.read_file(G3_out).to_crs("ESRI:102022")
WSFE_ADM = gpd.read_file(WSFE_out).to_crs("ESRI:102022")
print(GRID3_ADM.info(), "\n\n", GRID3_ADM.sample(10), "\n\n", GRID3_ADM.crs, "\n\n", 
      WSFE_ADM.info(), "\n\n", WSFE_ADM.sample(10), "\n\n", WSFE_ADM.crs, "\n\n", 
      GRID3_ADM['gridcode'].max(), WSFE_ADM['gridcode'].max())

In [ ]:
# Split serial back into separate dataset fields.
# For example, Burkina: WSFE and ADM: 4+3=7 digits. GRID3 and ADM: 6+3=9 digits.
G3_Fill = len_G3 + len_ADM
WSFE_Fill = len_WSFE + len_ADM

GRID3_ADM['gridstring'] = GRID3_ADM['gridcode'].astype(str).str.zfill(G3_Fill)
WSFE_ADM['gridstring'] = WSFE_ADM['gridcode'].astype(str).str.zfill(WSFE_Fill)

GRID3_ADM['Sett_ID'] = GRID3_ADM['gridstring'].str[:-len_ADM].astype(int) # Remove the last 3 digits to get the GRID3 portion.
GRID3_ADM['ADM_ID'] = GRID3_ADM['gridstring'].str[-len_ADM:].astype(int) # Keep only the last 3 digits to get the ADM portion.
WSFE_ADM['year'] = WSFE_ADM['gridstring'].str[:-len_ADM].astype(int)
WSFE_ADM['ADM_ID'] = WSFE_ADM['gridstring'].str[-len_ADM:].astype(int)

print(GRID3_ADM.sample(10), WSFE_ADM.sample(10))

In [ ]:
# Dissolve any features that have the same G3 and ADM values so that we have a single unique feature per settlement.
# Note: we do NOT want to dissolve the WSFE features. Distinct features for noncontiguous builtup areas of the same year is necessary to separate them in the Near tool step.

print(time.ctime())
GRID3_ADM = GRID3_ADM.dissolve(by=['Sett_ID', 'ADM_ID'], as_index=False)
print(GRID3_ADM.info(), GRID3_ADM.head(), "\n\n", time.ctime())

In [ ]:
# Remove features where year, settlement, or admin area = 0.
# This was supposed to be resolved earlier with the gdal_calc NoDataValue parameter. Being thorough.

print("Before: WSFE %s and GRID3 %s\n" % (WSFE_ADM.shape, GRID3_ADM.shape))
WSFE_ADM = WSFE_ADM.loc[(WSFE_ADM["year"] != 0) & (WSFE_ADM["ADM_ID"] != 0)] # Since we change the datatype to integer, no need to include all digits. Otherwise, it would need to be: != '0000'
GRID3_ADM = GRID3_ADM.loc[(GRID3_ADM["Sett_ID"] != 0) & (GRID3_ADM["ADM_ID"] != 0)]
print("After: WSFE %s and GRID3 %s\n" % (WSFE_ADM.shape, GRID3_ADM.shape))

In [ ]:
# The Bounded_ID is our new unique settlement identifier for subsequent matching steps.
GRID3_ADM['Bounded_ID'] = GRID3_ADM.index
WSFE_ADM['WSFE_ID'] = WSFE_ADM.index
GRID3_ADM = GRID3_ADM[['Sett_ID', 'Bounded_ID', 'ADM_ID', 'geometry']]
WSFE_ADM = WSFE_ADM[['WSFE_ID', 'year', 'ADM_ID', 'geometry']]

In [ ]:
# Validation: 
# The first two printed numbers should be the same. There shouldn't be any GRID3 rows with matching Sett_ID and ADM_IDs.
# The latter two numbers should be different, and the first should be larger. We never dissolved WSFE by any column.

print(len(GRID3_ADM[['Sett_ID', 'ADM_ID']]),
      len(GRID3_ADM[['Sett_ID', 'ADM_ID']].drop_duplicates()),
      len(WSFE_ADM[['year', 'ADM_ID']]),
      len(WSFE_ADM[['year', 'ADM_ID']].drop_duplicates()))

In [ ]:
GRID3_ADM.to_file(
    driver='GPKG', filename=os.path.join(Intermediate,'GRID3_ADM.gpkg'), layer='GRID3_ADM_cleaned')
WSFE_ADM.to_file(
    driver='GPKG', filename=os.path.join(Intermediate,' WSFE_ADM.gpkg'), layer='WSFE_ADM_cleaned')

---

## 4. UNIQUE SETTLEMENTS FROM WSFE AND GRID3: TWO VERSIONS

Note that there are 2 versions here, so that we can create a fragmentation index:
1. **Boundless, aka boundary-agnostic settlements**: Unique settlements are linked to GRID3 settlement IDs. Administrative areas do not influence the extents of the settlement.
2. **Bounded, aka politically-defined settlements**: Settlements in the Boundless dataset which spread across more than one administrative area are split into separate settlements in the Bounded dataset. The largest polygon after the split is considered the "principal" settlement, and polygons in other admin areas are considered "fragments." By dividing the fragment area(s) of the Bounded settlement by the area of the Boundless settlement, we can acquire a fragmentation index for each locality.

### 4.1 BOUNDED SETTLEMENTS: Near Join by ADM group.

In [ ]:
print("Number of admin areas with GRID3 features: %s" % len(GRID3_ADM['ADM_ID'].unique().tolist()))
print("Number of admin areas with WSFE features: %s" % len(WSFE_ADM['ADM_ID'].unique().tolist()))
print("Number of admin areas where one dataset is observed but the other is not: %s" % (
    len(GRID3_ADM['ADM_ID'].unique().tolist()) - len(WSFE_ADM['ADM_ID'].unique().tolist())))

In [ ]:
ADM_IDs = sorted(GRID3_ADM['ADM_ID'].unique().tolist())
ADM_IDs

In [ ]:
# We're creating this field to help in removing duplicates from the sjoin_nearest, next section.
GRID3_ADM['G3_Area'] = GRID3_ADM['geometry'].area / 10**6

In [ ]:
# Create empty geodataframe to append onto using the dataframe whose geometry we want to retain.
Bounded = GRID3_ADM[0:0]
Bounded["year"] = pd.Series(dtype='int')
Bounded.info()

In [ ]:
for ID in ADM_IDs:
    WSFE_shard = WSFE_ADM.loc[WSFE_ADM['ADM_ID'] == ID]
    GRID3_shard = GRID3_ADM.loc[GRID3_ADM['ADM_ID'] == ID]
    WSFE_GRID3_shard = gpd.sjoin_nearest(WSFE_shard, 
                                         GRID3_shard, 
                                         how='inner',
                                         max_distance=500)
    Bounded = pd.concat([Bounded, WSFE_GRID3_shard])
    print('Completed near join in admin area %s. %s \n' % (ID, time.ctime()))
print('Completed near join for all ADMs. %s \n' % time.ctime())

del WSFE_shard, GRID3_shard, WSFE_GRID3_shard

In [ ]:
Bounded.sample(20)

In [ ]:
Bounded.info()

In [ ]:
# Remove WSFE features that did not match any GRID3 settlements.
Bounded = Bounded.loc[~Bounded['Sett_ID'].isna()]
Bounded.info()

In [ ]:
del GRID3_ADM, ADM_IDs

### 4.2 Remove duplicates: where buildup polygons intersected with more than one GRID3 settlement extent.
This happens when the first dataset (WSFE) intersects (distance = 0) with more than one feature of the second dataset (GRID3). More common for large cities. For example, Yaoundé, CMN has a large contiguous 1985 WSFE polygon which overlaps several small GRID3 features that are not Yaoundé.

In [ ]:
# The first number should always be zero. 
# The second tells us whether/how many WSFE polygons were duplicated by the Near join.

print(len(WSFE_ADM[WSFE_ADM.duplicated('WSFE_ID')]), len(Bounded[Bounded.duplicated('WSFE_ID')]))

In [ ]:
# If there are duplicate WSFE_IDs, then we need to choose between them.
# We'll pick the one that joined with the largest GRID3 polygon.
# To do that, we can just sort the dataframe by GRID3 areas, then drop_duplicates. 
# It will retain the first row of each WSFE_ID group.
Bounded = Bounded.sort_values('G3_Area', ascending=False).drop_duplicates(['WSFE_ID'])
Bounded.info()

In [ ]:
print(len(Bounded[Bounded.duplicated('WSFE_ID')]))

In [ ]:
# Now we can dissolve with the WSFE years, now that we can group them by their administratively split ID.
Bounded = Bounded.dissolve(by=['year', 'Bounded_ID'], as_index=False)
print(Bounded.info(), Bounded.sample(10))

In [ ]:
# Clean up and save to file.
Bounded = Bounded[['ADM_ID_left', 'year', 'Bounded_ID', 'Sett_ID', 'geometry']].rename(columns={"ADM_ID_left": "ADM_ID"})
Bounded = Bounded.astype({"ADM_ID":'int', "Bounded_ID":'int', "Sett_ID":'int', "year":'int'})
print(Bounded.sample(10))

In [ ]:
Bounded.to_file(
    driver='GPKG', filename=os.path.join(Intermediate,'NonCumulativeSettlements.gpkg'), layer='Settlements_Bounded')

In [ ]:
del WSFE_ADM

### 4.3 BOUNDLESS SETTLEMENTS: Dissolve features that were split by an ADM boundary.

In [ ]:
# Fragments of any bounded settlement will be combined into a single "boundless" settlement in this version.
# It is based on their "Sett_ID", which is a direct loan from the GRID3 settlement features.
Boundless = Bounded.dissolve(by=['year', 'Sett_ID'], as_index=False)
print(Boundless.info(), Boundless.sample(10))

In [ ]:
# Clean up and save to file.
Boundless.to_file(driver='GPKG', 
                  filename=os.path.join(Intermediate,'NonCumulativeSettlements.gpkg'), layer='Settlements_Boundless')

---

## 5. CUMULATIVE ANNUALIZED SETTLEMENT EXTENTS
DISSOLVE BY YEAR SETS: Create separate feature layers of each cumulative year.

### 5.1 Define study years for each for loop.

In [ ]:
Boundless = gpd.read_file(os.path.join(Intermediate,'NonCumulativeSettlements.gpkg'), layer='Settlements_Boundless')

In [ ]:
ReversedStudyYears_to2014 = []
for i in AllStudyYears:
    ReversedStudyYears_to2014.insert(0,i)
ReversedStudyYears_to2014.remove(2015)
print('\n\n', ReversedStudyYears_to2014)

### 5.2 Starting with main Boundless dataset, create a cumulative area feature layer for each year.

In [ ]:
# For each year in the growth stats study, we are taking features from all years prior to and including that year, 
# dissolving those features, and exporting as its own file.

for item in AllStudyYears:
    print('Subsetting to cumulative area for year: %s. %s\n' % (item, time.ctime()))
    YearSet = Boundless[Boundless['year'].between(
        1985, item, inclusive=True)] # Inclusive parameter means we include the years 1985 and "item" rather than only between them.
    print('Dissolving so that each unique settlement (Sett_ID) has a single cumulative WSFE feature. %s\n' % time.ctime())
    YearDissolve = YearSet.dissolve(by='Sett_ID', 
                                        aggfunc={"year": "max", "ADM_ID":"min"}, # Though ADM_ID should be matching every time.
                                        as_index=False)
    print('Write to file. %s\n' % time.ctime())
    YearName = ''.join(['Cu', str(item), '_Boundless'])
    YearDissolve.to_file(driver='GPKG', filename=os.path.join(Results,'CumulativeSettlements.gpkg'), layer=YearName)
    del YearSet, YearDissolve
print("Done with all years in set. %s" % time.ctime())

##### Join area information from each cumulative layer onto the latest year dataset.

In [ ]:
# The latest year in the study contains all settlements. Merge all other years' areas onto this dataset.
SettAreas = gpd.read_file(os.path.join(Results,'CumulativeSettlements.gpkg'), layer=
                          ''.join(['Cu', str(2015), '_Boundless'])) 
SettAreas['AREA2015'] = SettAreas['geometry'].area / 10**6
SettAreas = pd.DataFrame(SettAreas).drop(columns='geometry') # We have settlement IDs, so no need to join spatially!

In [ ]:
for item in ReversedStudyYears_to2014:
    print("Loading cumulative layer for year %s. %s\n" % (item, time.ctime()))
    YearLayer = gpd.read_file(os.path.join(Results,'CumulativeSettlements.gpkg'), 
                              layer=''.join(['Cu', str(item), '_Boundless']))
    print("Adding area field and converting to non-spatial dataframe. %s\n" % (time.ctime()))
    AreaYearName = ''.join(['AREA', str(item)])
    YearLayer[AreaYearName] = YearLayer['geometry'].area/ 10**6 
    YearLayer = pd.DataFrame(YearLayer)[['Sett_ID', AreaYearName]]
    print("Merging variables from %s onto our latest year (%s) via table join. %s\n" % (item, 2015, time.ctime()))
    SettAreas = SettAreas.merge(YearLayer, how='left', on='Sett_ID')
print("Done merging annualized areas onto latest year geometries. Saving to file. %s\n" % (time.ctime()))


print(SettAreas.info())
SettAreas.to_csv(os.path.join(Results, 'Area.csv'))

In [ ]:
del SettAreas

### 5.3 Repeat for Bounded dataset.

In [ ]:
# Bounded = gpd.read_file(r'Results/NonCumulativeSettlements.gpkg', layer='Settlements_Bounded')

for item in AllStudyYears:
    print('Subsetting to cumulative area for year: %s. %s\n' % (item, time.ctime()))
    YearSet = Bounded[Bounded['year'].between(1985, item, inclusive=True)] # Inclusive parameter means we include the years 1985 and "item" rather than only between them.
    print('Dissolving so that each unique settlement (Bounded_ID) has a single cumulative WSFE feature. %s\n' % time.ctime())
    YearDissolve = YearSet.dissolve(by='Bounded_ID', 
                                        aggfunc={"year": "max", "ADM_ID":"min", "Sett_ID":"min"}, # Though ADM_ID and Sett_ID should be matching every time.
                                        as_index=False)
    print('Write to file. %s\n' % time.ctime())
    YearName = ''.join(['Cu', str(item), '_Bounded'])
    YearDissolve.to_file(driver='GPKG', filename=os.path.join(Results,'CumulativeSettlements.gpkg'), layer=YearName)
    del YearSet, YearDissolve
print("Done with all years in set. %s" % time.ctime())

In [ ]:
SettAreas = gpd.read_file(os.path.join(Results,'CumulativeSettlements.gpkg'), 
                          layer=''.join(['Cu', str(2015), '_Bounded']))
SettAreas['AREA2015'] = SettAreas['geometry'].area / 10**6
SettAreas = pd.DataFrame(SettAreas).drop(columns='geometry')

In [ ]:
for item in ReversedStudyYears_to2014:
    print("Loading cumulative layer for year %s. %s\n" % (item, time.ctime()))
    YearLayer = gpd.read_file(os.path.join(Results,'CumulativeSettlements.gpkg'), 
                              layer=''.join(['Cu', str(item), '_Bounded']))
    print("Adding area field and converting to non-spatial dataframe. %s\n" % (time.ctime()))
    AreaYearName = ''.join(['AREA', str(item)])
    YearLayer[AreaYearName] = YearLayer['geometry'].area/ 10**6 
    YearLayer = pd.DataFrame(YearLayer)[['Bounded_ID', AreaYearName]]
    print("Merging variables from %s onto our latest year (%s) via table join. %s\n" % (item, 2015, time.ctime()))
    SettAreas = SettAreas.merge(YearLayer, how='left', on='Bounded_ID')
print("Done merging annualized areas onto latest year geometries. Saving to file. %s\n" % (time.ctime()))

print(SettAreas.info())
SettAreas.to_csv(os.path.join(Results, 'Area_Bounded.csv'))

In [ ]:
del SettAreas

### 5.4 One settlement geofile to rule them all. ...and in the Sett_ID bind them.
The annualized values can be stored as distinct non-spatial dataframes. Their Sett_IDs will be used to join onto this geoversion with place names for the summary stats.

In [ ]:
Settlements = gpd.read_file(os.path.join(Results,'CumulativeSettlements.gpkg'), 
                           layer=''.join(['Cu', str(2015), '_Boundless']))[['Sett_ID', 'ADM_ID', 'geometry']]
print(Settlements.info())
print(Settlements.crs)
Settlements.to_file(driver='GPKG', 
                       filename=os.path.join(Results,'SETTLEMENTS.gpkg'), 
                       layer='SETTLEMENTS_equalarea')

In [ ]:
# Saving all the final products as WGS84.
Settlements_WGS = Settlements.to_crs(4326) 
print(Settlements_WGS.info())
print(Settlements_WGS.crs)
Settlements_WGS.to_file(driver='GPKG', 
                       filename=os.path.join(Results,'SETTLEMENTS.gpkg'), 
                       layer='SETTLEMENTS')

### 5.5 Buffer the area of the Boundless dataset's latest year to mask raster data in later sections.
The Bounded dataset would also be fine for our purposes here. The buffer is dissolved to a single feature to be used for its total extents, which are identical between Bounded & Boundless datasets.

In [ ]:
# Create buffer layer(s) to use as maximum distance for Near joins.

# Population buffer: 2km
Distance = 2000 # The Africa Albers projection is in meters. Saving in this projection to use in later sections.

print('Creating buffer layer. %s' % time.ctime())
BufferLayer = Settlements[['Sett_ID', 'geometry']]
BufferLayer['geometry'] = BufferLayer['geometry'].apply(
    make_valid).buffer(Distance) # make_valid is a workaround for any null geometries.
print('Finished buffer layer creation. %s' % time.ctime())

BufferFileName1 = ''.join(['Buff', str(Distance), 'm_', str(2015)])
BufferLayer.to_file(driver='GPKG', filename=os.path.join(Intermediate,'Catchment.gpkg'), layer=BufferFileName1)
print('Saved to file. %s' % time.ctime())

In [ ]:
# Nighttime Lights buffer: 250m
Distance = 250

print('Creating buffer layer. %s' % time.ctime())
BufferLayer = Settlements[['Sett_ID', 'geometry']]
BufferLayer['geometry'] = BufferLayer['geometry'].apply(
    make_valid).buffer(Distance) # make_valid is a workaround for any null geometries.
print('Finished buffer layer creation. %s' % time.ctime())

BufferFileName2 = ''.join(['Buff', str(Distance), 'm_', str(2015)])
BufferLayer.to_file(driver='GPKG', filename=os.path.join(Intermediate,'Catchment.gpkg'), layer=BufferFileName2)
print('Saved to file. %s' % time.ctime())

---

## 6. PLACE NAMES
Join urban place names from UCDB, Africapolis, and GeoNames onto the settlement vectors.

### 6.1 Load placename datasets, filter, and project.

In [ ]:
# Anytime we use a spatial join or work with area, 
# my preference is to keep it in a planar, equal area, meters projection. So we'll load as the Africa Albers.
Settlements = gpd.read_file(os.path.join(Results, 'SETTLEMENTS.gpkg'), layer='SETTLEMENTS_equalarea')
Settlements['AREA2015'] = Settlements['geometry'].area / 10**6

# Load, pull name field, rename, and reproject to match the catchments CRS.
UCDB = gpd.read_file(os.path.join(Source, 'PlaceName', 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg'), 
                     layer=0)[['UC_NM_MN', 'geometry']].rename(
    columns={"UC_NM_MN": "UCDB_Name"}).to_crs("ESRI:102022")

Africapolis = gpd.read_file(os.path.join(Source, 'PlaceName', 'AFRICAPOLIS2020.shp'))[['agglosName', 'geometry']].rename(
    columns={"agglosName": "Afpl_Name"}).to_crs("ESRI:102022")

GeoNames = gpd.read_file(os.path.join(Source, 'PlaceName', 'GeoNames.gpkg'), 
                         layer=0)[['GeoName', 'geometry']].to_crs("ESRI:102022")

print(Settlements.info(), UCDB.info(), Africapolis.info(), GeoNames.info())

### 6.2 Join placenames onto settlements geodataframe.

In [ ]:
# We wrap it in pd.DataFrame() since the sjoin() is the last time we need the geometry.

GeoNames = pd.DataFrame(gpd.sjoin_nearest(GeoNames, Settlements, 
                             how='left', distance_col="distGN", max_distance=250, 
                             lsuffix="G3", rsuffix="GN")).drop(columns='geometry')
Africapolis = pd.DataFrame(gpd.sjoin_nearest(Africapolis, Settlements, 
                             how='left', distance_col="distAF", max_distance=250,
                             lsuffix="G3", rsuffix="Af")).drop(columns='geometry')
UCDB = pd.DataFrame(gpd.sjoin_nearest(UCDB, Settlements, 
                             how='left', distance_col="distUC", max_distance=250,
                             lsuffix="G3", rsuffix="UC")).drop(columns='geometry')

In [ ]:
print(GeoNames.info())
print(Africapolis.info())
print(UCDB.info())

In [ ]:
alldatasets = [pd.DataFrame(Settlements).drop(columns='geometry'),
               Africapolis[['Sett_ID', 'Afpl_Name', 'distAF']], 
               GeoNames[['Sett_ID', 'GeoName', 'distGN']],
               UCDB[['Sett_ID', 'UCDB_Name', 'distUC']]]

SettlementsNamed = reduce(lambda left,right: pd.merge(left,right,on=['Sett_ID'], how='left'), alldatasets)
SettlementsNamed[['Afpl_Name', 'GeoName', 'UCDB_Name']] = SettlementsNamed[['Afpl_Name', 'GeoName', 'UCDB_Name']].fillna('UNK')

# Replace NaN values with a countable distance.
SettlementsNamed[['distAF', 'distGN', 'distUC']] = SettlementsNamed[['distAF', 'distGN', 'distUC']].fillna(-1)

In [ ]:
print(SettlementsNamed.info())
print(SettlementsNamed.sample(10))

In [ ]:
del UCDB, Africapolis, GeoNames

The near joins should have prevented duplication of rows, but if df1 intersects with two features in df2, it creates a new row. Two of our placenames sources are polygons, so there may be instances.

In [ ]:
SettlementsNamed[SettlementsNamed.duplicated('Sett_ID', keep=False)]

In [ ]:
SettlementsNamed.drop_duplicates(subset=['Sett_ID'], inplace=True, keep='first')
SettlementsNamed.info() # Range of entries should be the same as original Settlements file.

### 6.3 Reduce to single name column.

In [ ]:
# Determine which source has a name geometrically closest to the settlement.
# Since we switched NaN values to -1 earlier, we also resolved what happens in the event of a tie, 
# i.e. when more than one source is 0.0 meters from the settlement. It will take the value from the first column.
SettlementsNamed['SettName'] = "UNK"
SettlementsNamed['closest'] = SettlementsNamed[['distAF', 'distGN', 'distUC']].idxmax(axis=1)

In [ ]:
SettlementsNamed.sample(20)

In [ ]:
# Create a single name column where non-named settlements are "UNK" but all others use one of the three name sources.
SettlementsNamed.loc[
    SettlementsNamed['closest'] == "distAF", 
    'SettName'] = SettlementsNamed['Afpl_Name']

SettlementsNamed.loc[
    SettlementsNamed['closest'] == "distUC", 
    'SettName'] = SettlementsNamed['UCDB_Name']

SettlementsNamed.loc[
    SettlementsNamed['closest'] == "distGN", 
    'SettName'] = SettlementsNamed['GeoName']

In [ ]:
SettlementsNamed.sample(20)

In [ ]:
SettlementsNamed[SettlementsNamed['SettName'] != 'UNK'].sample(20)

### 6.4 Make sure place name is unique by stripping smaller localities of duplicated names.

In [ ]:
Dupes = SettlementsNamed[ 
    (SettlementsNamed['SettName'] != 'UNK') & 
    (SettlementsNamed.duplicated('SettName', keep=False)) ] # keep=False is necessary to retain *all* duplicates, not just first or last in each group.

print("Number of named settlements: %s" % SettlementsNamed['SettName'].str.contains('UNK').value_counts()[False])
print("Number of named settlements where name is duplicated at least once: %s" % len(Dupes))

In [ ]:
Largest = Dupes.loc[Dupes.groupby(["SettName"])["AREA2015"].idxmax()]
print(Largest)

In [ ]:
# Filter to settlements which have a duplicated name and are not the largest of those with that name, then replace with UNK.
SettlementsNamed.loc[(~SettlementsNamed.Sett_ID.isin(Largest.Sett_ID)) 
                     & (SettlementsNamed.Sett_ID.isin(Dupes.Sett_ID)), 
                     'SettName'] = 'UNK'

In [ ]:
# Second number should now be zero.

print("Number of named settlements: %s" % SettlementsNamed['SettName'].str.contains('UNK').value_counts()[False])
print("Number of named settlements where name is duplicated at least once: %s" % len(SettlementsNamed[ 
    (SettlementsNamed['SettName'] != 'UNK') & 
    (SettlementsNamed.duplicated('SettName', keep=False)) ]))

In [ ]:
print(SettlementsNamed.info(), SettlementsNamed[SettlementsNamed['SettName'] != "UNK"].sample(20))

In [ ]:
# Drop extra columns and save to file.
SettlementsNamed = SettlementsNamed[['Sett_ID', 'SettName']]
SettlementsNamed.to_csv(r'Results/PlaceNames.csv')

In [ ]:
del SettlementsNamed

---

## 7. CREATE FRAGMENTATION INDEX
We are determining what percentage of a settlement's area lies outside of its administrative zone each year.
The index is a range of 0 to 100, i.e. the percent of the settlement area which is fragmented.

For each Sett_ID:
((Area of Boundless settlement - Area of largest Bounded settlement feature) / Area of Boundless settlement) * 100

### 7.1 Load boundless and bounded cumulative settlements and clean.

In [ ]:
BoundlessAreas = pd.read_csv(os.path.join(Results, 'Area.csv'))
print('Loaded Boundless dataset, whose settlements will be used as the index of the Fragmentation Index dataset. %s' 
      % time.ctime())
print(BoundlessAreas.info())

BoundedAreas = pd.read_csv(os.path.join(Results, 'Area_Bounded.csv'))
print('Loaded Bounded dataset, which will factor into the fragmentation calculation. %s' % time.ctime())
print(BoundedAreas.info())

In [ ]:
BoundlessAreas = BoundlessAreas.loc[:, ~BoundlessAreas.columns.str.contains('Unnamed')]
BoundedAreas = BoundedAreas.loc[:, ~BoundedAreas.columns.str.contains('Unnamed')]

In [ ]:
LargestFragments = BoundedAreas.loc[BoundedAreas.groupby(["Sett_ID"])["AREA2015"].idxmax()] 
print(LargestFragments.info())
print("Filtered the Bounded dataset to only rows where latest year's area is largest for each Sett_ID. %s" % time.ctime())
LargestFragments.columns = LargestFragments.columns.str.replace('AREA', 'Largest')
LargestFragments = LargestFragments.drop(columns=['year', 'ADM_ID'])
print("Renamed columns to avoid duplication during merge, and dropped unnecessary columns. %s" % time.ctime())
FragIndices = BoundlessAreas.merge(LargestFragments, how='left', on='Sett_ID')
print(FragIndices.info())

In [ ]:
del BoundlessAreas, BoundedAreas, LargestFragments

### 7.2 Merge and run fragmentation calculation.

In [ ]:
for item in AllStudyYears:
    YY = str(item) # 4-digit year
    AreaYY = ''.join(["AREA", YY]) # The Boundless area variable name
    LargestYY = ''.join(['Largest', YY]) # The Bounded largest area variable name
    FragYY = ''.join(["Frag", YY]) # Name for the fragmentation index variable
    print("Created names for Year %s's variables and temporary objects. %s" % (item, time.ctime()))
    
    FragIndices[FragYY] = ((FragIndices[AreaYY] - FragIndices[LargestYY]) / FragIndices[AreaYY]) * 100
    FragIndices[FragYY] = (FragIndices[FragYY].fillna(0).replace([np.inf, -np.inf], 0)).astype('int')
    print("Calculated fragmentation index for year %s. %s" % (item, time.ctime()))

# Remove unnecessary columns.
FragIndices = FragIndices.loc[:, ~FragIndices.columns.str.startswith('Largest')]
FragIndices = FragIndices.loc[:, ~FragIndices.columns.str.startswith('AREA')]

print('Completed fragmentation index calculations for all years. %s' % time.ctime())
print(FragIndices.info())
print(FragIndices.sort_values(by='Frag2015', ascending=False).head(20))

In [ ]:
FragIndices = FragIndices.drop(columns=['year', 'ADM_ID'])
FragIndices.to_csv(os.path.join(Results, 'FragIndex.csv'))
print('Saved to file. %s' % time.ctime())

In [ ]:
del FragIndices

---

## 8. PREPARE YEARLY DATASETS: POPULATION
Can use this as a template for other annualized rasters

### 8.1 Reproject and reclassify with settlement buffer mask.
Reclassify so that we only need to work with cells within X distance of settlements.

In [ ]:
PopList = glob.glob(os.path.join(Source, 'Population', "*.tif"))
Settlements = gpd.read_file(os.path.join(Results, 'SETTLEMENTS.gpkg'), layer='SETTLEMENTS')
PopList

In [ ]:
BatchZonal(RasterFileList= PopList, Zones = Settlements, 
           KeepFields = ['Sett_ID'], 
           RasterDirectory = os.path.join(Source, 'Population'), 
           OutPath = os.path.join(Results, 'Population.csv'), 
           Statistics=['sum'], 
           NoDataVal = -99999, 
           Prefix = 'POP',
           Suffix = '', 
           DropStatName = True)

---

## 9. PREPARE YEARLY DATASETS: NIGHTTIME LIGHTS

### 9.1 Reclassify with settlement buffer mask.
Reclassify so that we only need to work with cells within X distance of settlements. The two NTL sources have already been reprojected in a separate script, and cropped to Central & Western Africa.

In [ ]:
D_avg = glob.glob(os.path.join(Source, 'NTL', "*D*avg.tif"))
V_avg = glob.glob(os.path.join(Source, 'NTL', "*V*avg.tif"))
D_cfc = glob.glob(os.path.join(Source, 'NTL', "*D*cfc.tif"))
V_cfc = glob.glob(os.path.join(Source, 'NTL', "*V*cfc.tif"))
Settlements = gpd.read_file(os.path.join(Results, 'SETTLEMENTS.gpkg'), layer='SETTLEMENTS')

print(D_avg, V_avg, D_cfc, V_cfc)

#### Nighttime lights from two different sources

In [ ]:
BatchZonal(RasterFileList= D_avg, Zones = Settlements, 
           KeepFields = ['Sett_ID'], 
           RasterDirectory = os.path.join(Source, 'NTL'), 
           OutPath = os.path.join(Results, 'NTL_DMSP.csv'), 
           Statistics=['sum', 'max', 'mean', 'median', 'std'], 
           NoDataVal = -99999, # For NTL, the actual NoData is a "soft NaN": 3.40282e+38. Just using pop's here.
           Prefix = 'NTL_D',
           Suffix = '', 
           DropStatName = False)

BatchZonal(RasterFileList= V_avg, Zones = Settlements, 
           KeepFields = ['Sett_ID'], 
           RasterDirectory = os.path.join(Source, 'NTL'), 
           OutPath = os.path.join(Results, 'NTL_VIIRS.csv'), 
           Statistics=['sum', 'max', 'mean', 'median', 'std'], 
           NoDataVal = -99999,
           Prefix = 'NTL_V',
           Suffix = '', 
           DropStatName = False)

#### Cloud-free coverage (confidence metric)

In [ ]:
BatchZonal(RasterFileList= D_cfc, Zones = Settlements, 
           KeepFields = ['Sett_ID'], 
           RasterDirectory = os.path.join(Source, 'NTL'), 
           OutPath = os.path.join(Results, 'NTL_DMSPcfc.csv'), 
           Statistics=['sum', 'count', 'mean', 'median', 'std'], 
           NoDataVal = -99999, 
           Prefix = 'NTL_D',
           Suffix = '_cfc', 
           DropStatName = False)

BatchZonal(RasterFileList= V_cfc, Zones = Settlements, 
           KeepFields = ['Sett_ID'], 
           RasterDirectory = os.path.join(Source, 'NTL'), 
           OutPath = os.path.join(Results, 'NTL_VIIRScfc.csv'), 
           Statistics=['sum', 'count', 'mean', 'median', 'std'], 
           NoDataVal = -99999,
           Prefix = 'NTL_V',
           Suffix = '_cfc', 
           DropStatName = False)

---

## 10. FLOOD EXPOSURE BY RETURN PERIOD

### 10.1 Calculate Expected Annual Depth (EAD) using exceedance probabilities of every flood return period.

##### Flood layers

In [ ]:
FloodFolder = os.path.join(Source, 'Flood')

In [ ]:
InRasters = os.listdir(FloodFolder)
InRasters

In [ ]:
Exceedances = []
    
for Raster in InRasters:
    InPath = os.path.join(SourceFolder, Raster)
    RP = re.sub('\D', '', Raster)[1:] # Get the return period
    NewFileName = Raster.replace('.tif', '_EXC.tif')
    OutPath = os.path.join(FloodFolder, NewFileName)
    
    Calc = "(1/" + RP + ")*A"

    calcShell(A=InPath, OutFile=OutPath, Calculation = Calc)
    Exceedances = Exceedances + [NewFileName]
    
print('Done with list. New flood set: %s' % Exceedances)

In [ ]:
# gdal_calc doesn't always take well to adding together a large number of files, so we'll do it in 2 batches.

Calc = 'A+B+C+D+E'
OutName = 'Batch1.tif'

A = os.path.join(FloodFolder, Exceedances[0])
B = os.path.join(FloodFolder, Exceedances[1])
C = os.path.join(FloodFolder, Exceedances[2])
D = os.path.join(FloodFolder, Exceedances[3])
E = os.path.join(FloodFolder, Exceedances[4])

calcShell(A=A, B=B, C=C, D=D, E=E,
          OutFile = os.path.join(FloodFolder, OutName), 
          Calculation = Calc)


In [ ]:
Calc = 'A+B+C+D+E+F'
OutName = 'FU_ExpectedAnnualDepth.tif'

A = os.path.join(FloodFolder, 'Batch1.tif')
B = os.path.join(FloodFolder, Exceedances[5])
C = os.path.join(FloodFolder, Exceedances[6])
D = os.path.join(FloodFolder, Exceedances[7])
E = os.path.join(FloodFolder, Exceedances[8])
F = os.path.join(FloodFolder, Exceedances[9])

calcShell(A=A, B=B, C=C, D=D, E=E, F=F,
          OutFile = os.path.join(FloodFolder, OutName), 
          Calculation = Calc)

### 10.2 Reclassify and resample flood data and buildup data in preparation for the impact calculation.

##### Reclassify flood as a binary: flooded / not-flooded

In [ ]:
InPath = os.path.join(FloodFolder, OutName)
OutPath = os.path.join(FloodFolder, 'FU_EAD_reclassed.tif')

[xsize, ysize, geotransform, geoproj, Z] = readRaster(InPath)

Z[Z<0.15] = 0 # Not-flooded category. This includes no data cells.
Z[Z>=0.15] = 1 # Flooded category. This includes permanent water bodies.

writeRaster(OutPath,geotransform,geoproj,Z)
InPath = OutPath = None

print('Finished reclassifying. %s' % time.ctime())

##### Buildup

In [ ]:
WSFE = 'WSFE_equalarea.tif'
WSFEPath = os.path.join(Workspace, 'Buildup', WSFE)
OutPath = os.path.join(FloodFolder, WSFE.replace('equalarea.tif', 'simplified.tif'))

[xsize, ysize, geotransform, geoproj, Z] = readRaster(WSFEPath)

np.putmask(Z, Z>0, Z-1984) # All years now converted to at most 2 digits: 1-31. (All non-buildup = 0)

writeRaster(OutPath,geotransform,geoproj,Z)

print('\nSimplified buildup file: %s' % OutPath)

##### Resample flood to match buildup

In [ ]:
WSFEPath = os.path.join(FloodFolder, 'WSFE_simplified.tif') 

RasterPath = os.path.join(FloodFolder, 'FU_EAD_reclassed.tif')
OutPath = os.path.join(FloodFolder, 'FU_EAD_resampled.tif')
resampleRaster(RasterPath, WSFEPath, OutPath)
    
print('Resampled to match WSFE. %s' % time.ctime())

### 10.3 Mask out built areas that were not flooded.

In [ ]:
# WSFEPath = os.path.join(FloodFolder, 'WSFE_simplified.tif') 
    
InPath = os.path.join(FloodFolder, 'FU_EAD_resampled.tif')
OutPath = os.path.join(FloodFolder, 'FU_EAD_WSFEimpact.tif')

calcShell(A=WSFEPath, B=InPath, OutFile=OutPath, Calculation="A*B", OutType=" --type=Byte")
    
print('Done. Only built-up cells that have been flooded remain.. %s' % time.ctime())

### 10.4 Join with Settlements via concatenation
Using the serial method, combine settlement IDs with 1) WSFE year cells and 2) the flooded-only WSFE year cells under each scenario.

##### Rasterize the settlements we've created.

In [ ]:
# WSFEPath = os.path.join(FloodFolder, 'WSFE_simplified.tif') 
OutSett = os.path.join(Results, 'Settlements2015_rasterized.tif')
Settlements = gpd.read_file(r'Results/SETTLEMENTS.gpkg', layer='SETTLEMENTS_equalarea')[['Sett_ID', 'geometry']]

len_WSFE = 2 # We already know that the WSFE years are reclassified to 1-31, i.e. a max of 2 digits.

In [ ]:
ShapeToRaster(Shapefile=Settlements, ValueVar="Sett_ID", MetaRasterPath=WSFEPath, OutFilePath=OutSett, NewDType = 'uint32')

##### Calculations

In [ ]:
Calc = "(A*" + str(10**len_WSFE) + ")+B" 

FloodImpactPath = os.path.join(FloodFolder, 'FU_EAD_WSFEimpact.tif')
FloodSerialPath = os.path.join(FloodFolder, 'FU_Settlements_serial.tif')

#WSFEPath = os.path.join(FloodFolder, 'WSFE_simplified.tif')
WSFESerialPath = os.path.join(FloodFolder, 'WSFE_Settlements_serial.tif')

In [ ]:
calcShell(A=OutSett, B=FloodImpactPath, OutFile=FloodSerialPath, Calculation=Calc)
calcShell(A=OutSett, B=WSFEPath, OutFile=WSFESerialPath, Calculation=Calc)

### 10.5 Vector math to split raster strings into Settlement and WSFE year assignments.

##### Vectorize

In [ ]:
FloodVec = 'FloodedBuildup.shp' # Was having write issues when putting both in the same gpkg, so we're settling for .shp.
FloodVecPath = os.path.join(FloodFolder, FloodVec)
BuildVec = 'AllBuildup.shp' 
BuildVecPath = os.path.join(FloodFolder, BuildVec)

In [ ]:
RasterToShapefile(InRasterPath=FloodSerialPath, OutFilePath=FloodVecPath, 
                  OutName='', VariableName='gridcode')
RasterToShapefile(InRasterPath=WSFESerialPath, OutFilePath=BuildVecPath, 
                  OutName='', VariableName='gridcode')

##### Split string into separate fields

In [ ]:
Sett_rio = rasterio.open(os.path.join(Results, 'Settlements2015_rasterized.tif')).read(1)
len_Sett = len(str(Sett_rio.max()))
Sett_rio = None

Fill = len_Sett + 2 # Add the digits stored as len_WSFE # or just write +2 since we already know the length of reclassed WSFE.

OutPackage = os.path.join(FloodFolder, 'FloodedSettlements.gpkg')

In [ ]:
# Load newly created vectorized datasets.
for File in [FloodVec, BuildVec]:
    InObject = gpd.read_file(os.path.join(FloodFolder, File)).to_crs("ESRI:102022")
    print(InObject.info(), '\n\n', InObject.sample(10), '\n\n', InObject.crs, '\n\n', InObject['gridcode'].max())
    
    InObject['gridstring'] = InObject['gridcode'].astype(str).str.zfill(Fill)

    InObject['Sett_ID'] = InObject['gridstring'].str[:-2].astype(int) # Remove the last digits to get the Sett ID portion.
    InObject['year'] = InObject['gridstring'].str[-2:].astype(int) # Keep only the last digits to get the year portion.
    InObject['year'] = np.where(InObject['year'] > 0, InObject['year'] + 1984, InObject['year']) # Reclass back to year value.
    
    print('%s Serial split by year of buildup and Sett ID.\n\n' % time.ctime(), InObject.sample(10))
    
    # Remove features where year or settlement = 0.
    print("%s Before: %s\n" % (File, InObject.shape))
    InObject = InObject.loc[(InObject["year"] >1984) & (InObject["year"] < 2016) & (InObject["Sett_ID"] != 0)] 
    print("%s After: %s\n" % (File, InObject.shape))

    # Save intermediate file.
    InObject.to_file(driver='GPKG', filename=OutPackage, layer=File.replace('.shp', ''))

### 10.6 Group by settlement and count cells for each year.

##### Flooded buildings

In [ ]:
Settlements = gpd.read_file(r'Results/SETTLEMENTS.gpkg', layer='SETTLEMENTS_equalarea')
AllSummaries = pd.DataFrame(Settlements).drop(columns='geometry')[['Sett_ID']]
Settlements = None

ValObject = pd.DataFrame(gpd.read_file(OutPackage, layer='FloodedBuildup'))[['Sett_ID', 'year']]

print(AllSummaries.info(), '\n', AllSummaries.sample(10), '\n', ValObject.info(), '\n', ValObject.sample(10))

In [ ]:
for BuiltYear in AllStudyYears:
    GroupedVals = ValObject[
        ValObject['year']<=BuiltYear].groupby(
        'Sett_ID', as_index=False)
    
    VariableName = ''.join(['FLDct_', str(BuiltYear)])
    
    AllSummaries = AllSummaries.merge(GroupedVals.count().rename(columns={'year': VariableName}), how = 'left', on='Sett_ID')

    print('\nDesired aggregation methods applied to settlement level, year %s. %s \n' % (BuiltYear, time.ctime()))

    # Save in-progress results
    AllSummaries.to_csv(os.path.join(FloodFolder, 'FloodedCellCount1999to2015.csv'))
    print(AllSummaries.sort_values(by=AllSummaries.columns[1], ascending=False).head(10))

##### All buildings

In [ ]:
Settlements = gpd.read_file(r'Results/SETTLEMENTS.gpkg', layer='SETTLEMENTS_equalarea')
AllSummaries = pd.DataFrame(Settlements).drop(columns='geometry')[['Sett_ID']]
Settlements = None
AllSummaries

ValObject = pd.DataFrame(gpd.read_file(OutPackage, layer='AllBuildup'))[['Sett_ID', 'year']]

In [ ]:
for BuiltYear in AllStudyYears:
    GroupedVals = ValObject[
        ValObject['year']<=BuiltYear].groupby(
        'Sett_ID', as_index=False)
    
    VariableName = ''.join(['BLDct_', str(BuiltYear)])
    
    AllSummaries = AllSummaries.merge(GroupedVals.count().rename(columns={'year': VariableName}), how = 'left', on='Sett_ID')

    print('\nDesired aggregation methods applied to settlement level, year %s. %s \n' % (BuiltYear, time.ctime()))

    # Save in-progress results
    AllSummaries.to_csv(os.path.join(FloodFolder, 'BuiltCellCount1999to2015.csv'))
    print(AllSummaries.sort_values(by=AllSummaries.columns[1], ascending=False).head(10))

### 10.7 Calculate area and percent flooded and save to file

In [ ]:
BuiltArea = pd.read_csv(os.path.join(FloodFolder, 'BuiltCellCount1999to2015.csv'))
Flood = pd.read_csv(os.path.join(FloodFolder, 'FloodedCellCount1999to2015.csv'))
Areas = pd.read_csv(os.path.join(Results, 'Areas1999to2015.csv'))

for Dataset in [BuiltArea, Flood, Areas]:
    if 'Unnamed: 0' in Dataset.columns:
        Dataset.drop(columns='Unnamed: 0', inplace=True)
    else:
        pass
    print(Dataset.info())

Stats = reduce(lambda  left,right: pd.merge(left,right,on=['Sett_ID'],
                                            how='outer'), [BuiltArea, Flood, Areas])
print(Stats.info())

In [ ]:
# Quick spot-checking. Number of flood cells should always be less than or equal to number of built area cells.
Check1 = (Stats['FLDct_2007'] > Stats['BLDct_2007']).sum()
Check2 = (Stats['FLDct_2000'] > Stats['BLDct_2000']).sum()
Check3 = (Stats['FLDct_2011'] > Stats['BLDct_2011']).sum()
print(Check1, Check2, Check3) # All should be zero. 

##### Percent flooded

In [ ]:
for year in AllStudyYears:
    RawVar = ''.join(['FLDct_', str(year)])
    DenomVar = ''.join(['BLDct_', str(year)])
    NewVar = ''.join(['FLDpc', str(year)])
    if ((RawVar in Stats.columns) and (DenomVar in Stats.columns)):
        Stats[NewVar] = Stats[RawVar] / Stats[DenomVar]
    else:
        pass
Stats.sort_values(by='FLDpc2005', ascending=False).head(10)

##### Area flooded

In [ ]:
for year in AllStudyYears:
    RawVar = ''.join(['FLDpc', str(year)])
    DenomVar = ''.join(['AREA', str(year)])
    NewVar = ''.join(['FLDarea', str(year)])
    if ((RawVar in Stats.columns) and (DenomVar in Stats.columns)):
        Stats[NewVar] = Stats[RawVar] * Stats[DenomVar]
    else:
        pass
Stats.sort_values(by='FLDarea2005', ascending=False).head(10)

In [ ]:
# Drop original variables.
Stats = Stats.loc[:, Stats.columns.str.contains('Sett|FLDpc|FLDarea')]
Stats.columns

In [ ]:
# Save to file
Stats.to_csv(os.path.join(Results, 'Flood1999to2015.csv'))

## 11. GROWTH STATISTICS

### 11.1 Load and prep.

In [ ]:
AllStudyYears = ListFromRange(1999, 2015)

In [ ]:
PlaceNames = pd.read_csv(os.path.join(Results, 'PlaceNames.csv'))
Areas = pd.read_csv(os.path.join(Results, 'Area.csv'))
Population = pd.read_csv(os.path.join(Results, 'Population.csv'))
DMSP = pd.read_csv(os.path.join(Results, 'NTL_DMSP.csv'))
VNL = pd.read_csv(os.path.join(Results, 'NTL_VIIRS.csv'))
#Flood = pd.read_csv(os.path.join(Results, 'Flood.csv'))

RawValues = [PlaceNames, Areas, Population, DMSP, VNL]

for Dataset in RawValues:
    if 'Unnamed: 0' in Dataset.columns:
        Dataset.drop(columns='Unnamed: 0', inplace=True)
    else:
        pass
    if 'year' in Dataset.columns:
        Dataset.drop(columns='year', inplace=True)
    else:
        pass
    print(Dataset.info(verbose=True))

In [ ]:
AllStats = reduce(lambda  left,right: pd.merge(left,right,on=['Sett_ID'],
                                            how='outer'), RawValues)
AllStats.to_csv(os.path.join(Results, 'AllStats.csv'))

AllStats[AllStats.SettName!='UNK'].sample(5)

### 11.2 Change over time of raw variables
pch = percent change

#### Population change

In [ ]:
Stats = PlaceNames.copy().merge(Population, how = 'outer', on='Sett_ID')
for year in AllStudyYears:
    RawVar = ''.join(['POP', str(year)])
    LagVar = ''.join(['POP', str(year-1)])
    NewVar = ''.join(['POPpch', str(year)])
    if ((RawVar in Stats.columns) and (LagVar in Stats.columns)):
        Stats[NewVar] = (Stats[RawVar] - Stats[LagVar]) / Stats[LagVar]
    else:
        pass

In [ ]:
# Drop original variables.
Stats = Stats.loc[:, Stats.columns.str.contains('Sett|pch')]
Stats.columns

In [ ]:
Stats.to_csv(os.path.join(Results, 'PopChange.csv'))
Stats.drop(columns='SettName', inplace=True)
AllStats = AllStats.merge(Stats, how='left', on='Sett_ID')
AllStats[AllStats.SettName!='UNK'].sort_values(by=AllStats.columns[5], ascending=False).head(10)

#### Area change

In [ ]:
Stats = PlaceNames.copy().merge(Areas, how = 'outer', on='Sett_ID')
for year in AllStudyYears:
    RawVar = ''.join(['AREA', str(year)])
    LagVar = ''.join(['AREA', str(year-1)])
    NewVar = ''.join(['AREApch', str(year)])
    if ((RawVar in Stats.columns) and (LagVar in Stats.columns)):
        Stats[NewVar] = (Stats[RawVar] - Stats[LagVar]) / Stats[LagVar]
    else:
        pass

In [ ]:
# Drop original variables.
Stats = Stats.loc[:, Stats.columns.str.contains('Sett|pch')]
Stats.columns

In [ ]:
Stats.to_csv(os.path.join(Results, 'AreaChange.csv'))
Stats.drop(columns='SettName', inplace=True)
AllStats = AllStats.merge(Stats, how='left', on='Sett_ID')
AllStats[AllStats.SettName!='UNK'].sort_values(by=AllStats.columns[5], ascending=False).head(10)

#### NTL change

In [ ]:
Stats = PlaceNames.copy().merge(NTL, how = 'outer', on='Sett_ID')
Sensors = ['D_', 'V_']
Methods = ['sum', 'avg', 'max']

for year in AllStudyYears:
    for Sensor in Sensors:
        for agg in Methods:
            RawVar = ''.join(['NTL', agg, Sensor, str(year)])
            LagVar = ''.join(['NTL', agg, Sensor, str(year-1)])
            NewVar = ''.join(['NTL', agg, '_pch', Sensor, str(year)])
            if ((RawVar in Stats.columns) and (LagVar in Stats.columns)):
                Stats[NewVar] = (Stats[RawVar] - Stats[LagVar]) / Stats[LagVar]
            else:
                pass

In [ ]:
# Drop original variables.
Stats = Stats.loc[:, Stats.columns.str.contains('Sett|pch')]
Stats.columns

In [ ]:
Stats.to_csv(os.path.join(Results, 'NTLChange.csv'))

In [ ]:
Stats.drop(columns='SettName', inplace=True)
AllStats = AllStats.merge(Stats, how='left', on='Sett_ID')
AllStats[AllStats.SettName!='UNK'].sort_values(by=AllStats.columns[5], ascending=False).head(10)

#### Flood change: change in area and change in percent of total area

In [ ]:
Stats = PlaceNames.copy().merge(Flood, how = 'outer', on='Sett_ID')
for year in AllStudyYears:
    RawVar = ''.join(['FLDarea', str(year)])
    LagVar = ''.join(['FLDarea', str(year-1)])
    NewVar = ''.join(['FLDareapch', str(year)])
    if ((RawVar in Stats.columns) and (LagVar in Stats.columns)):
        Stats[NewVar] = (Stats[RawVar] - Stats[LagVar]) / Stats[LagVar]
    else:
        pass

In [ ]:
for year in AllStudyYears:
    RawVar = ''.join(['FLDpc', str(year)])
    LagVar = ''.join(['FLDpc', str(year-1)])
    NewVar = ''.join(['FLDpcpch', str(year)])
    if ((RawVar in Stats.columns) and (LagVar in Stats.columns)):
        Stats[NewVar] = (Stats[RawVar] - Stats[LagVar]) / Stats[LagVar]
    else:
        pass

In [ ]:
# Drop original variables.
Stats = Stats.loc[:, Stats.columns.str.contains('Sett|FLDareapch|FLDpcpch')]
Stats.columns

In [ ]:
Stats.to_csv(os.path.join(Results, 'FloodChange.csv'))
Stats.drop(columns='SettName', inplace=True)
AllStats = AllStats.merge(Stats, how='left', on='Sett_ID')
AllStats[AllStats.SettName!='UNK'].sort_values(by=AllStats.columns[5], ascending=False).head(10)

#### Update parent spreadsheet

In [ ]:
AllStats.info(verbose=True)

In [ ]:
AllStats.to_csv(os.path.join(Results, 'AllStats.csv'))

### 11.3 Densities
POPden = people per square kilometer
<br>NTL...den = nighttime light luminosity per square kilometer
<br>NTL...pop = nighttime light luminosity per capita

#### Population Density

In [ ]:
Stats = PlaceNames.copy().merge(Population, how = 'outer', on='Sett_ID')
Stats = Stats.merge(Areas, how='left', on='Sett_ID')

for year in AllStudyYears:
    RawVar = ''.join(['POPsum', str(year)])
    DenomVar = ''.join(['AREA', str(year)])
    NewVar = ''.join(['POPden', str(year)])
    if ((RawVar in Stats.columns) and (DenomVar in Stats.columns)):
        Stats[NewVar] = Stats[RawVar] / Stats[DenomVar]
    else:
        pass

In [ ]:
# Change in density
for year in AllStudyYears:
    RawVar = ''.join(['POPden', str(year)])
    LagVar = ''.join(['POPden', str(year-1)])
    NewVar = ''.join(['POPdenpch', str(year)])
    if ((RawVar in Stats.columns) and (LagVar in Stats.columns)):
        Stats[NewVar] = (Stats[RawVar] - Stats[LagVar]) / Stats[LagVar]
    else:
        pass

In [ ]:
# Drop original variables.
Stats = Stats.loc[:, ~Stats.columns.str.contains('AREA|sum|ct|year|ADM')]
Stats.columns

In [ ]:
Stats.to_csv(os.path.join(Results, 'PopDensity.csv'))
Stats.drop(columns='SettName', inplace=True)
AllStats = AllStats.merge(Stats, how='left', on='Sett_ID')
AllStats[AllStats.SettName!='UNK'].sort_values(by=AllStats.columns[5], ascending=False).head(10)

#### Nighttime Lights Density

In [ ]:
Stats = PlaceNames.copy().merge(NTL, how = 'outer', on='Sett_ID')
Stats = Stats.merge(Areas, how='left', on='Sett_ID')
Sensors = ['D_', 'V_']
Methods = ['sum', 'avg', 'max']

for year in AllStudyYears:
    for Sensor in Sensors:
        for agg in Methods:
            RawVar = ''.join(['NTL', agg, Sensor, str(year)])
            DenomVar = ''.join(['AREA', str(year)])
            NewVar = ''.join(['NTL', agg, '_den', Sensor, str(year)])
            if ((RawVar in Stats.columns) and (DenomVar in Stats.columns)):
                Stats[NewVar] = Stats[RawVar] / Stats[DenomVar]
            else:
                pass

In [ ]:
# Change in density
for year in AllStudyYears:
    for Sensor in Sensors:
        for agg in Methods:
            RawVar = ''.join(['NTL', agg, '_den', Sensor, str(year)])
            LagVar = ''.join(['NTL', agg, '_den', Sensor, str(year-1)])
            NewVar = ''.join(['NTL', agg, '_denpch', Sensor, str(year)])
            if ((RawVar in Stats.columns) and (LagVar in Stats.columns)):
                Stats[NewVar] = (Stats[RawVar] - Stats[LagVar]) / Stats[LagVar]
            else:
                pass

In [ ]:
list(Stats.columns)

In [ ]:
# Drop original variables.
Stats = Stats.loc[:, Stats.columns.str.contains('Sett|den')]
list(Stats.columns)

In [ ]:
Stats.to_csv(os.path.join(Results, 'NTLDensity.csv'))
Stats.drop(columns='SettName', inplace=True)
AllStats = AllStats.merge(Stats, how='left', on='Sett_ID')
AllStats[AllStats.SettName!='UNK'].sort_values(by=AllStats.columns[5], ascending=False).head(10)

#### Nighttime Lights per Capita

In [ ]:
Stats = PlaceNames.copy().merge(NTL, how = 'outer', on='Sett_ID')
Stats = Stats.merge(Population, how='left', on='Sett_ID')
Sensors = ['D_', 'V_']
Methods = ['sum', 'avg', 'max']

for year in AllStudyYears:
    for Sensor in Sensors:
        for agg in Methods:
            RawVar = ''.join(['NTL', agg, Sensor, str(year)])
            DenomVar = ''.join(['POPsum', str(year)])
            NewVar = ''.join(['NTL', agg, '_pop', Sensor, str(year)])
            if ((RawVar in Stats.columns) and (DenomVar in Stats.columns)):
                Stats[NewVar] = Stats[RawVar] / Stats[DenomVar]
            else:
                pass

In [ ]:
# Change in density
for year in AllStudyYears:
    for Sensor in Sensors:
        for agg in Methods:
            RawVar = ''.join(['NTL', agg, '_pop', Sensor, str(year)])
            LagVar = ''.join(['NTL', agg, '_pop', Sensor, str(year-1)])
            NewVar = ''.join(['NTL', agg, '_poppch', Sensor, str(year)])
            if ((RawVar in Stats.columns) and (LagVar in Stats.columns)):
                Stats[NewVar] = (Stats[RawVar] - Stats[LagVar]) / Stats[LagVar]
            else:
                pass

In [ ]:
list(Stats.columns)

In [ ]:
# Drop original variables.
Stats = Stats.loc[:, Stats.columns.str.contains('Sett|_pop')]
list(Stats.columns)

In [ ]:
Stats.to_csv(os.path.join(Results, 'NTLperCapita.csv'))
Stats.drop(columns='SettName', inplace=True)
AllStats = AllStats.merge(Stats, how='left', on='Sett_ID')
AllStats[AllStats.SettName!='UNK'].sort_values(by=AllStats.columns[5], ascending=False).head(10)

#### Update parent spreadsheet

In [ ]:
AllStats.info(verbose=True)

In [ ]:
AllStats.to_csv(os.path.join(Results, 'AllStats.csv'))

### 11.4 Urban Type

In [ ]:
for year in AllStudyYears:
    PopVar = ''.join(['POPsum', str(year)])
    DenVar = ''.join(['POPden', str(year)])
    NewVar = ''.join(['UrbType', str(year)])
    if ((PopVar in AllStats.columns) and (DenVar in AllStats.columns)):
        AllStats[NewVar] = 'LD'
        AllStats.loc[(AllStats[PopVar] >= 5000) & (AllStats[DenVar] >= 300), NewVar] = 'SDurban'
        AllStats.loc[(AllStats[PopVar] >= 50000) & (AllStats[DenVar] >= 1500), NewVar] = 'HDurban'
    else:
        pass

In [ ]:
AllStats.to_csv(os.path.join(Results, 'AllStats.csv'))

In [ ]:
Stats = AllStats.loc[:, AllStats.columns.str.contains('Sett|UrbType')]
list(Stats.columns)

In [ ]:
Stats[Stats.SettName!='UNK'].sort_values(by=Stats.columns[5], ascending=False).head(10)

In [ ]:
Stats.to_csv(os.path.join(Results, 'UrbanType.csv'))